# Fine-Tuning Qwen-Chat Large Language Model (Multiple GPUs)

Tongyi Qianwen is a large language model developed by Alibaba Cloud based on the Transformer architecture, trained on an extensive set of pre-training data. The pre-training data is diverse and covers a wide range, including a large amount of internet text, specialized books, code, etc. In addition, an AI assistant called Qwen-Chat has been created based on the pre-trained model using alignment mechanism.

This notebook uses Qwen-1.8B-Chat as an example to introduce how to fine-tune the Qianwen model using Deepspeed.

## Environment Requirements

Please refer to **requirements.txt** to install the required dependencies.

## Preparation

### Download Qwen-1.8B-Chat

First, download the model files. You can choose to download directly from ModelScope.

In [1]:
!ls

finetune_fullparameter_multi_gpu.ipynb	 finetune_qlora_single_gpu.ipynb
finetune_fullparameter_single_gpu.ipynb  readme.md
finetune_lora_multi_gpu.ipynb		 requirements.txt
finetune_lora_single_gpu.ipynb		 slurm_files
finetune_qlora_multi_gpu.ipynb


In [3]:
from modelscope.hub.snapshot_download import snapshot_download
model_dir = snapshot_download('Qwen/Qwen-1_8B-Chat', cache_dir='.', revision='master')

/data/jp_rivera/miniconda3/envs/zion/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-12-22 01:48:39,830 - modelscope - WARNING - Using branch: master as version is unstable, use with caution
Fetching 24 files:   0%|          | 0/24 [00:00<?, ?it/s]






























Fetching 24 files:   4%|▍         | 1/24 [00:01<00:25,  1.10s/it]







Fetching 24 files:   8%|▊         | 2/24 [00:01<00:11,  1.88it/s]




























Fetching 24 files:  29%|██▉       | 7/24 [00:01<00:02,  8.13it/s]




























Fetching 24 files:  42%|████▏     | 10/24 [00:02<00:02,  4.79it/s]































Fetching 24 files:  50%|█████     | 12/24 [00:02<00:02,  5.38it/s]
























Fetching 24 files:  62%|██████▎   | 15/24 [00:03<00:02,  4.45it/s]








Fetching 24 files:  67%|██████▋   | 16/24 [00:03<00:01,  4.65it/s]














Fetching 24 files:  75%|███████▌  | 18/24 [00:03<00:01,  5.42it/s]



Fetching 24 files:  79%|███████▉  | 19/24 [00:04<00:00,  5.22it/s]
















Fetching 24 files:  88%|████████▊ | 21/24 

### Download Example Training Data

Download the data required for training; here, we provide a tiny dataset as an example. It is sampled from [Belle](https://github.com/LianjiaTech/BELLE).

Disclaimer: the dataset can be only used for the research purpose.

In [4]:
!wget https://atp-modelzoo-sh.oss-cn-shanghai.aliyuncs.com/release/tutorials/qwen_recipes/Belle_sampled_qwen.json

--2024-12-22 01:49:33--  https://atp-modelzoo-sh.oss-cn-shanghai.aliyuncs.com/release/tutorials/qwen_recipes/Belle_sampled_qwen.json
Resolving atp-modelzoo-sh.oss-cn-shanghai.aliyuncs.com (atp-modelzoo-sh.oss-cn-shanghai.aliyuncs.com)... 47.101.88.43
Connecting to atp-modelzoo-sh.oss-cn-shanghai.aliyuncs.com (atp-modelzoo-sh.oss-cn-shanghai.aliyuncs.com)|47.101.88.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228189 (223K) [application/json]
Saving to: ‘Belle_sampled_qwen.json’

Belle_sampled_qwen. 100%[===================>] 222.84K   391KB/s    in 0.6s    

2024-12-22 01:49:34 (391 KB/s) - ‘Belle_sampled_qwen.json’ saved [228189/228189]



You can also refer to this format to prepare the dataset. Below is a simple example list with 1 sample:

```json
[
  {
    "id": "identity_0",
    "conversations": [
      {
        "from": "user",
        "value": "你好"
      },
      {
        "from": "assistant",
        "value": "我是一个语言模型，我叫通义千问。"
      }
    ]
  }
]
```

You can also use multi-turn conversations as the training set. Here is a simple example:

```json
[
  {
    "id": "identity_0",
    "conversations": [
      {
        "from": "user",
        "value": "你好，能告诉我遛狗的最佳时间吗？"
      },
      {
        "from": "assistant",
        "value": "当地最佳遛狗时间因地域差异而异，请问您所在的城市是哪里？"
      },
      {
        "from": "user",
        "value": "我在纽约市。"
      },
      {
        "from": "assistant",
        "value": "纽约市的遛狗最佳时间通常在早晨6点至8点和晚上8点至10点之间，因为这些时间段气温较低，遛狗更加舒适。但具体时间还需根据气候、气温和季节变化而定。"
      }
    ]
  }
]
```

## Fine-Tune the Model

You can directly run the prepared training script to fine-tune the model. **nproc_per_node** refers to the number of GPUs used fro training.

In [ ]:
!torchrun --nproc_per_node 2 --nnodes 1 --node_rank 0 --master_addr localhost --master_port 6601 ../../finetune.py \
    --model_name_or_path "Qwen/Qwen-1_8B-Chat/" \
    --data_path "Belle_sampled_qwen.json" \
    --bf16 True \
    --output_dir "output_qwen" \
    --num_train_epochs 5 \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 16 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 1000 \
    --save_total_limit 10 \
    --learning_rate 1e-5 \
    --weight_decay 0.1 \
    --adam_beta2 0.95 \
    --warmup_ratio 0.01 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --report_to "none" \
    --model_max_length 512 \
    --gradient_checkpointing True \
    --lazy_preprocess True \
    --deepspeed "../../finetune/ds_config_zero2.json"

## Test the Model

We can test the model as follows:

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig

tokenizer = AutoTokenizer.from_pretrained("output_qwen", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    "output_qwen",
    device_map="auto",
    trust_remote_code=True
).eval()

response, history = model.chat(tokenizer, "你好", history=None)
print(response)

New model Qwen2 0.5B

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B-Instruct")

# Define the directory where you want to save the model and tokenizer
save_directory = "/data/public_models/Qwen2-0.5B-Instruct"

# Save the tokenizer
tokenizer.save_pretrained(save_directory)

# Save the model
model.save_pretrained(save_directory)

[2024-12-27 01:28:59,124] [WARNING] [real_accelerator.py:181:get_accelerator] Setting accelerator to CPU. If you have GPU or other accelerator, we were unable to detect it.
[2024-12-27 01:28:59,127] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cpu (auto detect)


/data/jp_rivera/miniconda3/envs/zion/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


In [ ]:
#qwen2 7B

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-7B-Instruct")

# Define the directory where you want to save the model and tokenizer
save_directory = "/data/public_models/Qwen2-7B-Instruct"

# Save the tokenizer
tokenizer.save_pretrained(save_directory)

# Save the model
model.save_pretrained(save_directory)

Loading checkpoint shards: 100%|██████████| 4/4 [00:25<00:00,  6.26s/it]


# testing out dataset size

In [1]:
from dataclasses import dataclass, field
import json
import math
import logging
import os
from typing import Dict, Optional, List
import torch
from torch.utils.data import Dataset
from deepspeed import zero
from deepspeed.runtime.zero.partition_parameters import ZeroParamStatus

import transformers
from transformers import Trainer, GPTQConfig
import transformers.integrations.deepspeed as deepspeed
from transformers.trainer_pt_utils import LabelSmoother
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from accelerate.utils import DistributedType
from datasets import load_dataset, load_from_disk

import wandb

[2025-01-09 07:50:49,916] [WARNING] [real_accelerator.py:181:get_accelerator] Setting accelerator to CPU. If you have GPU or other accelerator, we were unable to detect it.
[2025-01-09 07:50:49,920] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cpu (auto detect)


/data/jp_rivera/miniconda3/envs/zion/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/data/jp_rivera/miniconda3/envs/zion/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
full_dataset = load_dataset("togethercomputer/RedPajama-Data-1T-Sample", split="train")
len(full_dataset)

930514

In [3]:
full_dataset[0]

{'text': "\\section{Introduction}\n\\label{sec:intro}\n\n\\emph{Gender diversity}, or more often its lack thereof, among participants to\nsoftware development activities has been thoroughly studied in recent years. In\nparticular, the presence of, effects of, and countermeasures for \\emph{gender\n  bias} in Free/Open Source Software (FOSS) have received a lot of attention\nover the past decade~\\cite{david2008fossdevs, qiu2010kdewomen,\n  nafus2012patches, kuechler2012genderfoss, vasilescu2014gender,\n  oneil2016debiansurvey, robles2016womeninfoss, terrell2017gender,\n  zacchiroli2021gender}.  \\emph{Geographic diversity} is on the other hand the\nkind of diversity that stems from participants in some global activity coming\nfrom different world regions and cultures.\n\nGeographic diversity in FOSS has received relatively little attention in scholarly\nworks. In particular, while seminal survey-based and\npoint-in-time medium-scale studies of the geographic origins of FOSS\ncontributo

In [ ]:
#The end